In [23]:
# First let's read in our dataset.

import numpy as np
import pandas as pd
df = pd.read_csv('news.csv')
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [24]:
# Removing the first column as it is unnecessary.

df = df.drop(df.columns[0],axis=1)
df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [25]:
# Combining the title and text into one column.

df['news'] = df['title'] + ' ' + df['text']
df.head()

,title,text,label,news
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,You Can Smell Hillary’s Fear Daniel Greenfield...
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,Watch The Exact Moment Paul Ryan Committed Pol...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,Kerry to go to Paris in gesture of sympathy U....
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,Bernie supporters on Twitter erupt in anger ag...
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,The Battle of New York: Why This Primary Matte...


In [26]:
# Converting the REAL/FAKE labels to a binary classification.

convert_to_binary = {'REAL':1,'FAKE':0}
df['label'] = df['label'].map(convert_to_binary)
df.head()

,title,text,label,news
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0,You Can Smell Hillary’s Fear Daniel Greenfield...
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0,Watch The Exact Moment Paul Ryan Committed Pol...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1,Kerry to go to Paris in gesture of sympathy U....
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0,Bernie supporters on Twitter erupt in anger ag...
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1,The Battle of New York: Why This Primary Matte...


In [27]:
# We can now drop the first two columns.

df = df.drop([df.columns[0],df.columns[1]],axis=1)
df.head()

,label,news
0,0,You Can Smell Hillary’s Fear Daniel Greenfield...
1,0,Watch The Exact Moment Paul Ryan Committed Pol...
2,1,Kerry to go to Paris in gesture of sympathy U....
3,0,Bernie supporters on Twitter erupt in anger ag...
4,1,The Battle of New York: Why This Primary Matte...


In [28]:
# Reindex our columns.

df = df.reindex(columns=['news','label'])
df.head()

,news,label
0,You Can Smell Hillary’s Fear Daniel Greenfield...,0
1,Watch The Exact Moment Paul Ryan Committed Pol...,0
2,Kerry to go to Paris in gesture of sympathy U....,1
3,Bernie supporters on Twitter erupt in anger ag...,0
4,The Battle of New York: Why This Primary Matte...,1



We now have a dataframe consisting of input-output pairs.

Text pre-processing
Next we need to pre-process our text. There are several natural language processing techniques which we will utilise.

In [29]:

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re

# Creating a list of stopwords (common words which are not important to the overall meaning of the article).
stop_words = stopwords.words('english')
stop_words.extend(['the','it','in'])

# Lemmatization is the process of reducing words to their basic form, e.g. dogs->dog, running->run.
WNL = WordNetLemmatizer()

# We now iterate through each article in the dataset.
for index, row in df.iterrows():
    filtered_article = ''
    article = row['news']
    # Removing all punctuation using RegEx.
    article = re.sub(r'[^\w\s]', '', article)
    # Tokenization: changing the article from one continuous string to a list of tokens (words).
    words = [word.lower() for word in nltk.word_tokenize(article)]
    # Removing stopwords.
    words = [word for word in words if not word in stop_words]
    # Lemmatization and concatenating the words back into a single string.
    words_lemmatized = []
    for word in words:
        if word == 'us':
            words_lemmatized.append(word)
        else:
            words_lemmatized.append(WNL.lemmatize(word))
    filtered_article = " ".join([word for word in words_lemmatized])
    
    # Replace the article with the new filtered version.
    df.loc[index, 'news'] = filtered_article
    
df.head()

,news,label
0,smell hillary fear daniel greenfield shillman ...,0
1,watch exact moment paul ryan committed politic...,0
2,kerry go paris gesture sympathy us secretary s...,1
3,bernie supporter twitter erupt anger dnc tried...,0
4,battle new york primary matter primary day new...,1


Vectorization
Machine learning algorithms expect numerical feature vectors with a fixed size as input, rather than the raw text documents with variable length which we currently have. So we use a process called vectorization, where the text is mapped to a vector of real numbers.

We are going to use a simple but effective strategy called Bag of Words, where word occurences are the only feature of the text which shall be used. So other more complicated features, such as the relative position of words in the text, are ignored.

One option is to use a simple count vectorizer. This creates a vector for each article which consists of the word count for each word in that article. However this method would then give more weighting to the less relevant words which frequently occur and less weighting to the words which rarely occur, even though these words are probably more important to the classification of the text.
Instead, we can use a Tf-idf vectorizer. Tf stands for term frequency and idf stands for inverse document frequency. So the vectorizer reduces the weighting for words which are very common and increases the weighting for words which are less common in the overall corpus.

In [30]:
# First let's split our dataframe into an input and output list. Our corpus to vectorize is df_input.
df_input = df['news']
df_output = df['label']

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(df_input)
tf_idf_matrix

<6335x80967 sparse matrix of type '<class 'numpy.float64'>'
	with 1762247 stored elements in Compressed Sparse Row format>

In [10]:
print(tf_idf_matrix.toarray())
print(df_output.values)

labels = df_output.values
labels[1]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0 0 1 ... 0 1 1]


0

In [11]:
# Splitting the data into test data and train data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tf_idf_matrix, df_output.values, random_state=0)

In [12]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
Accuracy = logreg.score(x_test, y_test)

print(Accuracy*100)

/Users/gus/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


91.28787878787878


The resulting matrix is sparse, meaning that it contains mainly 0's. This is because, for example, an article consists of 100 unique words, whereas the entire corpus consists of 1000s of words, so most word counts will be 0.

<b>Deep Learning</b>

I shall first apply a CNN architecture to the preprocessed data:

In [13]:
# setting random seed (for consistent results & choosing operating system)

from numpy.random import seed
from tensorflow.random import set_seed
seed(0)
set_seed(0)
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [14]:
# installing and importanting neccessary modules (mainly Keras)

!pip install keras
!pip install tensorflow
import numpy as np
import matplotlib.pyplot as plt

In [15]:
# hot encoding the data (go from binary 0 or 1 to [1,0] and [0,1] respectively)

from keras.utils import to_categorical
print("example point's binary outcome:", labels[100])
labels = to_categorical(labels)
print("same example point's one hot encoded vector:", labels[100])

example point's binary outcome: 1
same example point's one hot encoded vector: [0. 1.]


In [16]:
# resplitting data with 30 % as testing data (actually unneccesary to redo)

from sklearn.model_selection import train_test_split

x = tf_idf_matrix.toarray()
x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size = 0.3)

In [17]:
print(np.shape(x_train))
np.shape(y_train)
print(y_train)

(4434, 80967)
[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [18]:
# scaling the training data features. Removes mean and makes the datas variance = 1. 
# learns the correct transformation on the training data and then applies to the test data.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train) 

x_test = scaler.transform(x_test)

In [19]:
# importing all components required in the following CNN
from keras.models import Sequential # going to use a sequential neural network
from keras.layers import Dense # type of layer
from keras.optimizers import SGD # optimisation technique (stochastic gradient descent) used to update the weights

In [20]:
print(x.shape) # 6335 different articles and 80967 different featuers per article
num_features = x.shape[1] # = 80967; the number of features there are
num_classes = len(np.unique(labels))# = 2; the number of outcomes there are
print(num_classes)

(6335, 80967)
2


In [21]:
x_train3, x_test3, y_train3, y_test3 = train_test_split(tf_idf_matrix, df_output, test_size=0.2, random_state=42)

# Split train 75:25 as 0.8*0.25=0.2
# So now have 60% train, 20% validation and 20% test.
x_train3, x_val3, y_train3, y_val3 = train_test_split(x_train3, y_train3, test_size=0.25, random_state=42)

In [22]:
model = Sequential()

model.add(Dense(32, input_dim=num_features, activation='relu'))(encoded)
model.add(Dense(16, activation='relu')) 
model.add(Dense(8, activation='relu')) 
model.add(Dense(16, activation='relu')) 
model.add(Dense(32, activation='relu')) 
model.add(Dense(num_classes, activation='relu')) 

model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01), metrics=['accuracy']) 
history = model.fit(x_train, y_train, epochs=25)

NameError: name 'encoded' is not defined

In [ ]:
from keras import layers

# This is the size of our encoded representations
encoding_dim = num_features  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(x_train)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(784, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(x_train, decoded)

In [ ]:
# building neural network

model = Sequential() # its a sequential NN

model.add(Dense(32, input_dim=num_features, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu',name='encoded'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_features, activation='relu'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01), metrics=['accuracy']) # build NN, specifying Loss using, how optimising weights & metric

history = model.fit(x_train, x_train, epochs=25)

In [ ]:
from keras import Model

encoder_output = model.get_layer('encoded').output
encoder = Model(model.input, encoder_output)

In [ ]:
np.shape(encoder.predict(x_train))

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test) # evaulating the model we've trained to the test data

In [ ]:
plt.plot(history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('loss')

In [ ]:
plt.plot(history.history['accuracy'])
plt.xlabel('epochs')
plt.ylabel('accuracy')

<b>K Nearest Neighbours (KNN) </b>

In [ ]:
# separate the labels from the data frame
label_column = df.loc[:,'label']
labels = label_column.values
print(labels)
df_output.values

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)

# Train the model using the training sets and test it on test data
model.fit(x_train, y_train)
model.score(x_test, y_test)*100

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)

# Train the model using the training sets
model.fit(tf_idf_matrix, labels)
model.score(tf_idf_matrix, labels)